## ECE 590 Final Project Report

Shiwen Chen, Yujing Ke, Yangdi Shen

#### Abstract

“Zestimate” is the model developed by Zillow to estimate the prices of real estates but missing data in Zillow datasets would affect the performance of it. This report examines the methods adopted for handling with missing data based on existing models. The data given contains information in 58 different aspects that might affect housing price, however, due to large number of missing values, the accuracy of predicting the error between the estimated housing price and the real selling price could be heavily affected. Therefore, different types of method was implemented and compared with their performance against each other. 


#### Introduction
Real estate purchase is usually one of the most important and expensive decisions that a family makes in daily life. However, pricing the real estates is a complicated process which requires massive data, precise algorithm and proper features. Zillow Group, or simply Zillow, is a online real estate database company founded in 2006 which attempted to solve this problem. They developed a estimation method called “Zestimates”. “Zestimates” are estimated home values based on 7.5 million statistical and machine learning models that analyze hundreds of data points on each property (Zillow, 2018). After 10 years’ development, “Zestimates” has been proven to be a successful method to predict house prices, with a median margin of error as low as 5% today.
Our research question originally came from a Kaggle competition named “Zillow Prize:Zillow’s Home Value Prediction” which helped push the accuracy of the Zestimate even further. We attempted to develop a statistical model with machine learning techniques to reduce overall prediction errors. This model could be considered as a complement to Zestimate. 


#### Background

According to Hagerty’s article, although the accuracy of Zestimate is decent most of the time, it could be up to 25% away from the actual selling price on 11% of observations. What’s worse, among 1000 transactions, 34 cases was off by more than fifty percent. Therefore, numerous researchers and data scientists have already thought about looking into this problem and find a better way to help Zillow with more accurate estimation. 

Since our problem of lowering estimation error problem is on Kaggle, there are a bunch of machine learning methods and techniques presented. For example, Andy Harless introduced widely adopted methods using XGBoost and LightGBM boosting algorithm. He proposed a combination of the predicted value of both algorithms with different weights. At the same time, Nikunj presented his process of dealing with missing value. 

A heatmap of correlations between variables is drawn, in order to identify which variables has extremely high correlation with each other. In his post, he treated the missing value of each variable differently: fill some with the most common value, fill some with the mean value, predict some missing value with KNN and remove fields that have values with 97% or more missing. In Gelman et all’s paper, they proposed an unique evaluation strategy that utilizes a systematic review of a large set of online sources for Zestimate. It is suggested that strategy like theirs may also be helpful for large-scale error detection and correction. 

With these established relevant solutions to this problem, we intended to improve the performance of existing models and get a better result. A straightforward way of evaluating the output result is uploading the produced result on Kaggle and see how the score compares to other teams on public and private leaderboard. In addition, cross validation is implemented so that we could further verify the hypothesis about which missing value handling method works better. 


#### Data

Our dataset is from Kaggle competition “Zillow Prize: Zillow’s Home Value Prediction”. The test dataset consists of 2985217 records with 58 independent variables. The training dataset consists of 90275 records with one more dependent variable, log error. We try to use the training data to fit our model and then predict log error. The features include 13 features about area (like basement area, finished square feet), 10 counts (like room, bathroom count), 9 geological (like latitude, longitude), 19 types (like property land use type, heating system type) and others.

![image](https://raw.githubusercontent.com/y-ke/CS590/master/WechatIMG25.png)
        <center>Figure 1 The Distribution of Missing Data in Variables<center>
    
The most significant challenge we faced is that our test dataset has a lot of missing data. In the graph above, we plot the number of missing data for each variable. We could see that all the columns except “parcel id” have missing data. In some columns, like “story type id”, we almost have no data at all. For some records, we don’t even know the latitude and longitude of the houses, which means that those records would provide us with little information. Therefore, as one approach, we will remove the variables which contains more than 70% of missing data, which has been illustrated by the red line in the graph.


![image](https://raw.githubusercontent.com/y-ke/CS590/master/WechatIMG24.png)
        <center>Figure 2 Correlations Plot for Variables<center>
We also check the correlations among the variables. The heatmap above shows the correlations among remaining variables. (already removed variables with too many missing data) We could interpret some information from it. Firstly, some of variables are basically the same things. For instance, “calculated bathroom count number” and “full bath count” have a correlation of 0.98. The “tax amount”, “land tax value” and “tax value” could be merged into one feature. However, the tree-based models we used have an innate feature of being robust to correlated features, so we don’t need to remove those features. Secondly, we notice that all the independent variables have low correlations with dependent variable, log error. Therefore, we should consider non-linear models instead of any linear models.


#### Methods

![image](https://raw.githubusercontent.com/y-ke/CS590/master/WechatIMG26.png)
<center>Figure 3 The Flow Chart of Our Methods<center>


###### 1.Fill in Missing Data


As mentioned earlier in the background section, there are some established method adopted by many data scientist on Kaggle that helps with not only this problem specifically but also other analytical problems in general. However, the imputation methods recommended by data scientist are not always reliable, in fact, sometimes patching the data up does not lead to a better result. Plus, in Gelman’s book it was suggested that imputation methods may bring about a bias to the data as well as the value that we are predicting. It is a very common consequence that might have negative impact on outliers due to our assumption of how would the missing values are distributed.
According to Gelman et all’s book,  Data Analysis Using Regression and Multilevel / Hierarchical Models, we should decide which approach to select depending on how the missing pattern is. The missingness could be completely at random, or missingness could depend on unobserved predictors, or missingness could depends on the missing value itself. Yet generally in practice, we usually try to include as many variables as possible in a model so that it is reasonable to assume that the missing is at random. 

The easiest and the most straightforward way is, without a doubt, trying to avoid the data with missing values or only select data with a minimal of missing data. Therefore, as we discussed earlier in the data section, we draw a critical line that separates the features with at least 70% values presented so that it could be advantageous for bias and precision.   

###### 1.1 Use Mean / Median / Mode values


In Gelman et all’s book, a multiple of types of filling missing value method are introduced. In this project, we mainly adopted three of them: replace missing values with mean, median and mode values respectively.

Replacing the missing value with mean of the observed values is possibly the simplest way among all these methods. Yet, this way of handling with missing data could severely affect the distribution for this variable, cause complications including underestimates of the standard deviation. Besides, this method distorts the relationship between the features by pushing estimates of the correlation closer to zero. Similarly, median value replacement would lead to the similar consequences. And mode value imputation, is to fill the missing values with the most frequent value among the  observations, would also add bias to data.


###### 1.2 treat NA as a class
Another category of filling NA values is attained by filling an Indicator variables for missingness of continuous predictors. In other words, if the missing value is categorical, we create a new indicator to mark all of them as a new class; if the missing value is continuous, we fill the empty entries as -1 so that it could be treated as an unique value. This approach is relatively popular in social sciences and it yield biased coefficient estimates for the other variables in the model since it “pulled” the slope to be the same across the missing data groups. However, this bias could be alleviated by adding interactions between an indicator for response and these variables with large number of missing values. 


###### 1.3 KNN

We also try to fill the missing data by applying a k-nearest neighbors algorithm to predict those missing data. As mentioned above, some of the records (over 11000) lack the most basic features like latitude and longitude. Therefore, we are not going to predict the missing features for those records because it is obviously impossible.
 
We tried to use three features, latitude, longitude and tax amount to of non-missing data to fit KNN classification and regression models. The reason why we choose these three features is that almost all the records have those features and they could represent the location and approximate economic value of a house. In other words, with KNN method, we could assume that if one house is very close to another and it has similar tax payment, then they could have some similarities in other features.

We fit the model to predict three columns of missing data, “calculated finished square feet”, “year built” and “assessed value of the built structure” (structure tax value). The most important reason for choosing those three is that the numbers of missing data in those three features are relatively small. Since KNN is a computational expensive method, we want to apply it on the features with fewer missing data. Also, we can find reasonable explanations for the similarities. For instance, if the locations and tax amounts are similar, we could assume two houses also have similar areas.

###### 2 . Tree boosting
The XGBoost stands for “Extreme Gradient Boosting” while the term “Gradient Boosting” came from Friedman’s paper about Greedy Function Approximation. The model of XGBoost is tree ensembles, which is a set of classification and regression trees. The picture below describes how this algorithm works in computer as if the classifier is like a human brain.

In this example, the tree 1 worked as an decision tree and at each step it used a specific criterion to divide the given group into two sub categories. Subsequently, a numerical value of how the different categories valued to is assigned. In a different tree, tree 2, a new aspect of criterion was given and assignment of different score is given. The prediction scores of each individual tree are added up for the final calculation and comparison. In practical, two trees are not strong enough and the industry usually use the tree ensemble model, which sums the prediction of multiple trees together. 

One feature of XGBoost is that it automatically treat missing value as a class, and this feature is called one-hot encoding. Here one hot encoding refers a process converting categorical variables into a form that could be provided to machine learning algorithms to perform better in prediction. For example, categories can be binarized, as in this category (with value 1) or not (with value 0) and then be used as features to train the model. 

![image](https://raw.githubusercontent.com/y-ke/CS590/master/d.png)
<center>Figure 4 Example of XGBoost<center>

However, we wanted to use more than one algorithm therefore at the same time we also considered the performance of LightGBM. The most evident disadvantage of XGBooster is that when given tons and tons of data, it could take a long time to train. While the LightGBM is a faster and distributed gradient boosting framework that uses decision tree algorithm and performances better than XGBoost in many cases. Similar to XGBoost, it splits the tree but unlike XGBoost doing splitting level-wisely, LightGBM splitted the tree leaf wise with the best fit. Consequently, the expanding on the leaf in LightGBM can reduce more loss than XGBoost algorithm and lead to superior accuracy. In addition, as the name suggests, the running time of LightGBM is exceedingly fast.

![image](https://raw.githubusercontent.com/y-ke/CS590/master/b.png)
<center>Figure 5 XGBoost and LightGBM Tree Growth Structure<center>
    
###### 3. Performance Assessment

The performance is assessed with the Mean absolute error of prediction:

$$ MAE = \frac{1}{m} \sum_{1}^{m} \mid y_{predict}^{i} - y_{real}^{i} \mid $$
And the log error is the logged difference between Zestimate and the actual selling price. As the test dataset is much larger than the training dataset so that we are not worrying about overfitting. In short, the final score assigned by Kaggle on leaderboard could work as a satisfactory indicator of the performance. Therefore, we would use Kaggle scores to compare the performance of different filling strategies.


#### Results

The baseline model is taken from the Kaggle kernel posted by Andy Harless where he set leave number as 512 in lightGBM. In the data processing phase, he only dropped two features with text values in them (property zoning description and property county land use code) and kept all the remaining 56 feature. In his model, the missing values are filled in with the median value of the feature. The MAE score of this model 0.0757628, also showed as the left-most bar in the figure 6 below.

![image](https://raw.githubusercontent.com/y-ke/CS590/master/c.png)
<center>Figure 6 Light GBM prediction performance with different data filling methods<center>
    
We tried different methods of filling the missing data and feature selection to improve the performance. Those methods include drop features with more than 70% of missing values, using mean / median / mode values to fill in missing values, treating NA as a different class, and using K-nearest neighbours to predict values based on other features.
 In figure 6 above, we can see that those methods can somewhat improve the performance. When working with all 56 features in the baseline model, treating NA as a class works better than filling in missing values with mean / median / mode values. It is also very effective to drop 28 features that have above 70% of missing values.
The best performance by our model is 0.075615. The improvement is very small in value. However, in the fierce Kaggle competition of Zillow's home value prediction, this amount of improvement in score can raise one's rank by 300 among the 3,800 participants. 

![image](https://raw.githubusercontent.com/y-ke/CS590/master/a.png)
<center>Figure 7 Prediction results and corresponding run time with light GBM and XGBooster<center>

As shown in figure 5 above, the light GBM method has slightly better performance than XGBooster but longer run time. Theoretically, the leaf-wise tree growth method is faster because it buckets continuous feature value into discrete bins. However, in our dataset, most of the features are already discrete or categorical values such as number of bedroom and bathroom, and type of architectural style. Also, to achieve similar results, more leaves need to be used by lightGBM than XGBooster. Therefore, the speed advantage of lightGBM didn't emerge in our project.


![image](https://raw.githubusercontent.com/y-ke/CS590/master/e.png)
<center>Figure 8 Relationship between predicted log error and true log error (in different scales)<center>
    
If we look at the predicted response and the true value, we will find that the predicted value has smaller order of magnitude than the true value. This means our model can only explain little proportion of the log error of Zestimate, which is also the case of all other models by the other participants in this Kaggle competition. As shown in figure 8 above, the distribution of the predicted value doesn't have very discernable pattern. The model performs relatively better on observations with larger positive log error. However, when log error is among -0.3 and 0.3, which contains most of observations, the predicted value looks more randomly distributed.
The reason for this seemingly poor performance is probably that Zestimate is powerful enough that leaves little space for improvement. The Zestimate tool is trained on similar features but much more observations over the years. The log error of Zestimate that we are trying to predict is likely just the noise. This noise may be explained by other features not presented in the current database, or may be simply irreducible. 
For this project, we are not going to interpret the model result for two reasons. Firstly, as illustrated in the last paragraph, the model failed to effectively predict the log error. Secondly, it makes less sense to discuss what feature contributes to the log error as the it is derived from the Zestimated model which we don't have access to.



#### Conclusions
In this project, we are trying to predict the log error made by Zestimate using 58 features and tree boosting algorithm. Built on many existing models presented in the kaggle competition, our main focus is on exploring way to handle missing value in order to improve the performance of model. We found that treating NA values as a class works well for this dataset. Also, discarding features with large proportion of missing values is a good thing to do in terms of improving the overall performance. However, this approach is not necessarily applicable to every dataset. If you are working on a different problem, we recommend you to go through different approaches presented in this report and explore a suitable way to fill in the missing values. 
This project still have a few limitations that should be addressed in the future study:
- We presented the use of K-nearest neighbours to predict missing values as one technique of filling missing data. However, we didn't explore very deep into it. This approach is subjective in that we need to manually determine which features to use as predictors. The number of predictors should not be too large as KNN suffers from high dimensionality, which lead to numerous possible combinations of features to choose from. Moreover, KNN is a computationally expensive algorithm that can take long run time to fill in missing value in our ~3,000,000-entry dataset. Therefore, we only made a few attempts on some features with less missing values. In future study, we can explore more possibilities and further tap the potential of this method.
- Among our 58 features, a few have large correlation with each other. However, we found out later that GBM-based models have an innate feature to assume uncorrelated inputs (Laurae, 2016), which can cause problems when using lightGBM on our dataset. To solve this problem, we present two direction of future study. First, to identify and remove redundant features. Secondly, to use XGBoost which is robust to correlated feature.  As suggested by Laurea (2016), for two perfectly correlated variables, XGBoost will pickup exactly 50%|50% of each feature.
- Ensemble method takes the weighted average result of various models as the final result for evaluation. Ensembling of two or more good models can usually outperform a single model (Dietterich & Thomas, 2000). In fact, we see many high ranking models in the Kaggle competition adopts this method. For future study, we can explore more on model ensembling techniques to improve the performance.


#### Roles

Shiwen: Implemented the missing data filling and tree boosting algorithm; Responsible for the results and conclusion parts of the report; helped with the video production

Yangdi: Visualize and preprocess the data; Implement KNN prediction on missing data; Responsible for writing data, some of the method parts of the report.

Yujing: wrote background and some of the method parts of the report; helped with the video visual


#### References

Andy Harless, “XGB w/o Outliers & LGB with Outliers Combined.” Accessed April 25, 2018. https://www.kaggle.com/aharless/xgb-w-o-outliers-lgb-with-outliers-combined.

Dietterich, Thomas G. "Ensemble methods in machine learning." International workshop on multiple classifier systems. Springer, Berlin, Heidelberg, 2000.

Friedman, Jerome H. “Greedy Function Approximation: A Gradient Boosting Machine.” The Annals of Statistics29, no. 5 (200110): 1189–1232. https://doi.org/10.1214/aos/1013203451.

Gelman, I. A., and N. Wu. “Combining Structured and Unstructured Information Sources for a Study of Data Quality: A Case Study of Zillow.Com.” In 2011 44th Hawaii International Conference on System Sciences, 1–12, 2011. https://doi.org/10.1109/HICSS.2011.115.


Hagerty, James R. “How Good Are Zillow’s Estimates?” WSJ. Accessed April 26, 2018. http://www.wsj.com/articles/SB117142055516708035.
Gelman, Andrew, and Jennifer Hill. “Data Analysis Using Regression and Multilevel/Hierarchical Models (Final Version: 5 July 2006),” n.d., 11.

Laurae. Ensembles of tree-based models: why correlated features do not trip them — and why NA matters. 2016. https://medium.com/data-design/ensembles-of-tree-based-models-why-correlated-features-do-not-trip-them-and-why-na-matters-7658f4752e1b

Nikunj, “Carefully Dealing with Missing Values.” Accessed April 25, 2018. https://www.kaggle.com/nikunjm88/carefully-dealing-with-missing-values.


Zillow. Zillow Prize: Zillow’s Home Value Prediction (Zestimate). https://www.kaggle.com/c/zillow-prize-1

XGBoost. Introduction to Boosted Trees. http://xgboost.readthedocs.io/en/latest/model.html
“Introduction to Boosted Trees — Xgboost 0.71 Documentation.” Accessed April 26, 2018. http://xgboost.readthedocs.io/en/latest/model.html.
